# Introduction to Newton Physics

[![ Click here to deploy.](https://brev-assets.s3.us-west-1.amazonaws.com/nv-lb-dark.svg)](https://brev.nvidia.com/launchable/deploy?launchableID=env-35QaaoiXx6VDmVNBOEtmpLs9VBs)

**Newton** is a GPU-accelerated physics simulation engine built upon [NVIDIA Warp](https://github.com/NVIDIA/warp), specifically targeting roboticists and simulation researchers.

This notebook provides an introduction to Newton's core concepts:
- Architecture and design philosophy
- Creating models with `ModelBuilder`
- Setting up and running solvers
- Visualizing simulations with `ViewerViser`

## Key Features

- **GPU-accelerated**: Leverages NVIDIA Warp for fast, scalable simulation
- **Multiple solver implementations**: XPBD, VBD, MuJoCo, Featherstone, SemiImplicit
- **Modular design**: Easily extendable with new solvers and components
- **Differentiable**: Supports differentiable simulation for machine learning
- **Rich Import/Export**: Load models from URDF, MJCF, USD, and more
- **Open Source**: Maintained by Disney Research, Google DeepMind, and NVIDIA


## Setup and Imports

First, let's import the necessary libraries:


In [ ]:
from pathlib import Path

import warp as wp
from pxr import Usd

import newton
import newton.examples
import newton.usd

## Newton Architecture

Newton follows a clear architectural pattern:

```
ModelBuilder → Model → State → Solver → Visualization
```

### Core Components

1. **ModelBuilder**: Constructs models from primitives or imported assets
   - Add bodies, shapes, joints, and constraints
   - Configure physical properties (mass, inertia, friction, etc.)
   - Import from URDF, MJCF, or USD files

2. **Model**: Encapsulates the physical structure and parameters
   - Immutable after finalization
   - Contains all bodies, shapes, joints, and their properties
   - Provides collision detection via `model.collide()`

3. **State**: Represents the dynamic state at a point in time
   - Positions, velocities, and forces
   - Mutable - updated by the solver
   - Can be saved/restored for replay or checkpointing

4. **Solver**: Advances the simulation by integrating physics
   - Multiple implementations: XPBD, VBD, MuJoCo, etc.
   - Takes current state and produces next state
   - Handles constraints, contacts, and forces

5. **Viewer**: Visualizes the simulation
   - Multiple backends: OpenGL, Rerun, USD, File recording
   - Real-time or offline rendering
   - Interactive debugging and inspection


## Building a Model with ModelBuilder

The `ModelBuilder` is the primary way to construct simulation scenes. Let's create a simple scene with various collision shapes falling onto a ground plane.

### Step 1: Create the ModelBuilder


In [ ]:
# Create a new model builder
builder = newton.ModelBuilder()

# Add a ground plane (infinite static plane at z=0)
builder.add_ground_plane()

### Step 2: Add Rigid Bodies with Collision Shapes

Bodies are the fundamental dynamic entities in Newton. Each body can have one or more collision shapes attached to it.

The workflow is:
1. Create a body with `add_body()` - returns a body index
2. Attach shapes to the body using `add_shape_*()` methods
3. Shapes inherit properties from the body and contribute to its mass/inertia


In [ ]:
# Height from which to drop shapes
drop_z = 2.0

# SPHERE
sphere_pos = wp.vec3(0.0, -2.0, drop_z)
body_sphere = builder.add_body(
    xform=wp.transform(p=sphere_pos, q=wp.quat_identity()),
    key="sphere",  # Optional: human-readable identifier
)
builder.add_shape_sphere(body_sphere, radius=0.5)

# CAPSULE
capsule_pos = wp.vec3(0.0, 0.0, drop_z)
body_capsule = builder.add_body(xform=wp.transform(p=capsule_pos, q=wp.quat_identity()), key="capsule")
builder.add_shape_capsule(body_capsule, radius=0.3, half_height=0.7)

# CYLINDER
cylinder_pos = wp.vec3(0.0, -4.0, drop_z)
body_cylinder = builder.add_body(xform=wp.transform(p=cylinder_pos, q=wp.quat_identity()), key="cylinder")
builder.add_shape_cylinder(body_cylinder, radius=0.4, half_height=0.6)

# BOX
box_pos = wp.vec3(0.0, 2.0, drop_z)
body_box = builder.add_body(xform=wp.transform(p=box_pos, q=wp.quat_identity()), key="box")
builder.add_shape_box(body_box, hx=0.5, hy=0.35, hz=0.25)

print(f"Added {builder.body_count} bodies with collision shapes")

### Step 3: Add a Mesh Body

Newton can also simulate arbitrary triangle meshes. Let's load a mesh from a USD file:


In [ ]:
# Load a mesh from a USD file
usd_stage = Usd.Stage.Open(newton.examples.get_asset("bunny.usd"))
demo_mesh = newton.usd.get_mesh(usd_stage.GetPrimAtPath("/root/bunny"))

# Add the mesh as a rigid body
mesh_pos = wp.vec3(0.0, 4.0, drop_z - 0.5)
body_mesh = builder.add_body(xform=wp.transform(p=mesh_pos, q=wp.quat(0.5, 0.5, 0.5, 0.5)), key="bunny")
builder.add_shape_mesh(body_mesh, mesh=demo_mesh)

print(f"Added mesh body with {demo_mesh.vertices.shape[0]} vertices")

### Step 4: Finalize the Model

Once all bodies and shapes are added, we finalize the model. This converts the Python data structures into GPU-optimized arrays and makes the model ready for simulation.


In [ ]:
# Finalize the model - this creates the simulation-ready Model object
model = builder.finalize()

print("Model finalized:")
print(f"  Bodies: {model.body_count}")
print(f"  Shapes: {model.shape_count}")
print(f"  Joints: {model.joint_count}")

## Creating States and Control

After finalizing the model, we need to create state objects to hold the simulation state:

- **State**: Holds positions, velocities, and forces
- **Control**: Holds control inputs (joint torques, motor commands, etc.)
- **Contacts**: Holds collision contact information

We typically need two state objects for time integration (current and next).


In [ ]:
# Create two state objects for time integration
state_0 = model.state()  # Current state
state_1 = model.state()  # Next state

# Create control object (for joint actuators, etc.)
control = model.control()

# Perform initial collision detection
contacts = model.collide(state_0)

print("State and control objects created")

## Setting Up the Solver

Newton provides multiple solver implementations. Here we'll use **XPBD** (Extended Position-Based Dynamics), which is:
- Fast and stable
- Good for rigid body dynamics
- Works well with complex contact scenarios

Other available solvers include:
- `SolverVBD`: Vellum-Based Dynamics (good for cloth/soft bodies)
- `SolverMuJoCo`: MuJoCo integration (high accuracy)
- `SolverFeatherstone`: Articulated body algorithm
- `SolverSemiImplicit`: Simple semi-implicit Euler


In [ ]:
# Create the XPBD solver with 10 constraint iterations
solver = newton.solvers.SolverXPBD(model, iterations=10)

print(f"Solver created: {type(solver).__name__}")

## Configuring the Simulation Loop

Let's set up the simulation parameters and create a simulation function:


In [ ]:
# Simulation parameters
fps = 60  # Frames per second for visualization
frame_dt = 1.0 / fps  # Time step per frame
sim_time = 0.0  # Current simulation time
sim_substeps = 10  # Number of physics substeps per frame
sim_dt = frame_dt / sim_substeps  # Physics time step

print("Simulation configured:")
print(f"  Frame rate: {fps} Hz")
print(f"  Frame dt: {frame_dt:.4f} s")
print(f"  Physics substeps: {sim_substeps}")
print(f"  Physics dt: {sim_dt:.4f} s")

### The Simulation Function

The core simulation loop follows this pattern:

1. Clear forces from the previous step
2. Apply external forces (gravity is automatic)
3. Detect collisions
4. Step the solver forward in time
5. Swap state buffers


In [ ]:
def simulate():
    """Run multiple physics substeps for one frame."""
    global state_0, state_1

    for _ in range(sim_substeps):
        # Clear accumulated forces
        state_0.clear_forces()

        # Detect collisions
        contacts = model.collide(state_0)

        # Advance the simulation by one physics timestep
        solver.step(state_0, state_1, control, contacts, sim_dt)

        # Swap states (next becomes current)
        state_0, state_1 = state_1, state_0

## GPU Acceleration with CUDA Graphs

For maximum performance on CUDA devices, we can capture the simulation loop as a CUDA graph. This reduces kernel launch overhead significantly:


In [ ]:
# Capture the simulation as a CUDA graph (if running on GPU)
if wp.get_device().is_cuda:
    with wp.ScopedCapture() as capture:
        simulate()
    graph = capture.graph
    print("CUDA graph captured for optimized execution")
else:
    graph = None
    print("Running on CPU (no CUDA graph)")

## Visualization with ViewerViser

Newton integrates with [Viser](https://github.com/nerfstudio-project/viser), a powerful browser-based 3D visualization library that provides:
- Real-time 3D rendering in WebGL
- Interactive camera controls
- Jupyter notebook integration
- Static HTML export and recording

The `ViewerViser` launches a web server and can be embedded in Jupyter notebooks or viewed in a browser.


In [ ]:
# Create the Viser viewer
recording_path = Path("../_static/recordings/00_introduction.viser").resolve()
recording_path.parent.mkdir(parents=True, exist_ok=True)
viewer = newton.viewer.ViewerViser(verbose=False, record_to_viser=str(recording_path))

# Set the model (this logs the static geometry)
viewer.set_model(model)

## Running the Simulation

Now let's run the simulation and visualize it! We'll simulate 500 frames (about 8 seconds at 60 fps).

The viewer will display:
- All rigid bodies with their shapes
- Contact points and normals
- Interactive 3D camera controls


In [ ]:
# Run the simulation
num_frames = 500

for _ in range(num_frames):
    # Execute the simulation (use CUDA graph if available)
    if graph:
        wp.capture_launch(graph)
    else:
        simulate()

    # Log the current state to the viewer
    viewer.begin_frame(sim_time)
    viewer.log_state(state_0)
    viewer.log_contacts(contacts, state_0)
    viewer.end_frame()

    # Advance simulation time
    sim_time += frame_dt

print(f"\nSimulation complete! Total time: {sim_time:.2f} seconds")

viewer

## Summary

In this tutorial, we covered the essential workflow for using Newton:

1. **ModelBuilder**: Create and configure the simulation scene
   - Add bodies with `add_body()`
   - Attach shapes with `add_shape_*()`
   - Finalize with `finalize()`

2. **Model & State**: Manage simulation data
   - Model is immutable after finalization
   - States hold dynamic quantities (positions, velocities)
   - Collision detection via `model.collide()`

3. **Solver**: Advance the physics simulation
   - Multiple solver options (XPBD, VBD, MuJoCo, etc.)
   - Step forward with `solver.step()`
   - GPU-accelerated with CUDA graphs

4. **Viewer**: Visualize and debug
   - Multiple backends (OpenGL, Viser, Rerun, USD, File)
   - Real-time or offline rendering
   - Interactive inspection and playback

## Next Steps

To learn more about Newton:

- **Examples**: Explore the `newton/examples/` directory for more complex scenarios
- **Documentation**: Visit [newton-physics.github.io](https://newton-physics.github.io/newton/)
- **Importing Models**: Learn to load URDF, MJCF, and USD files
- **Articulated Bodies**: Create robots with joints and actuators
- **Differentiable Simulation**: Use Newton for gradient-based optimization